Import potrebnih knjižnic:

In [ ]:
import pandas as pd
import numpy as np

1.1 Izbor podatkov

| Ime spremenljivke | Pomen / opis | Tip podatka |
|---|---|---|
| customerID | Enolična identifikacijska oznaka posamezne stranke. | Kategorna (ID string) |
| gender | Spol stranke (Male ali Female). | Kategorna (nominalna) |
| SeniorCitizen | Označuje, ali je stranka starejša oseba (1 = da, 0 = ne). | Numerična (binarna, lahko tudi kategorialna) |
| Partner | Ali ima stranka partnerja (Yes / No). | Kategorna (nominalna) |
| Dependents | Ali ima stranka odvisne člane (otroke, družinske člane) (Yes / No). | Kategorna (nominalna) |
| tenure | Število mesecev, kolikor je stranka naročena pri podjetju. | Numerična (diskretna) |
| PhoneService | Ali stranka uporablja telefonske storitve (Yes / No). | Kategorna (nominalna) |
| MultipleLines | Ali ima stranka več telefonskih linij (Yes, No, No phone service). | Kategorna (nominalna) |
| InternetService | Vrsta internetne storitve (DSL, Fiber optic, No). | Kategorna (nominalna) |
| OnlineSecurity | Dodatna storitev spletne varnosti (Yes, No, No internet service). | Kategorna (nominalna) |
| OnlineBackup | Storitve spletne varnostne kopije (Yes, No, No internet service). | Kategorna (nominalna) |
| DeviceProtection | Zaščita naprav (Yes, No, No internet service). | Kategorna (nominalna) |
| TechSupport | Tehnična podpora (Yes, No, No internet service). | Kategorna (nominalna) |
| StreamingTV | Storitve spletnega televizijskega pretakanja (Yes, No, No internet service). | Kategorna (nominalna) |
| StreamingMovies | Storitve spletnega pretakanja filmov (Yes, No, No internet service). | Kategorna (nominalna) |
| Contract | Vrsta pogodbe: Month-to-month, One year, Two year. | Kategorna (ordinalna) |
| PaperlessBilling | Ali stranka uporablja brezpapirno obračunavanje (Yes / No). | Kategorna (nominalna) |
| PaymentMethod | Način plačila (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)). | Kategorna (nominalna) |
| MonthlyCharges | Mesečni strošek naročnine (v USD). | Numerična (kontinuirna) |
| TotalCharges | Skupni strošek, ki ga je plačala stranka do danes. | Numerična (kontinuirna) |
| Churn | Ali je stranka prekinila naročnino (Yes / No). | Ciljna spremenljivka, kategorialna (binarna) |

1.2 Pregled in čiščenje podatkov

In [ ]:
df = pd.read_csv("Telco-Customer-Churn.csv")

print(df.info())
print(df.head())
print(df.describe(include="all"))

missing = df.isna().sum()
print("Manjkajoče vrednosti:\n", missing)

df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

df.loc[df["tenure"] == 0, "TotalCharges"] = 0

df["SeniorCitizen"] = df["SeniorCitizen"].map({0: "No", 1: "Yes"})

print("Manjkajoče po pretvorbi:\n", df.isna().sum())

for col in df.columns:
    if df[col].dtype in ["float64", "int64"]:
        df[col].fillna(df[col].median(), inplace=True)
    else:
        df[col].fillna(df[col].mode()[0], inplace=True)

duplicates = df.duplicated().sum()
print("Število podvojenih vrstic:", duplicates)

df = df.drop_duplicates()

numeric_cols = ["tenure", "MonthlyCharges", "TotalCharges"]

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = ((df[col] < lower) | (df[col] > upper)).sum()
    print(f"{col} – število ekstremnih vrednosti: {outliers}")

    df[col] = np.clip(df[col], lower, upper)

print(df.info())
print(df.describe())

df.to_csv("Telco-Customer-Churn-Cleaned.csv", index=False)


Pri spremenljivki TotalCharges so bile zaznane manjkajoče vrednosti, ki se pojavljajo izključno pri strankah z ničelno dobo trajanja naročniškega razmerja (tenure = 0). Ker te stranke še niso imele obračunanih stroškov, so bile manjkajoče vrednosti nadomeščene z vrednostjo 0, kar ohranja semantično pravilnost podatkov in preprečuje umetno zviševanje skupnih stroškov.

Spremenljivka SeniorCitizen je bila pretvorjena iz binarne oblike (0/1) v nominalno (Yes/No) zaradi boljše berljivosti in interpretacije pri opisni analizi.